## Python IG Web Scrapping Using Selenium and Access to Some API

In [1]:
# Requirements
# I'm using python 3.9.12 anaconda

# !pip install selenium==3.141.0
# !pip install webdriver-manager
# !pip install requests==2.30.0
# !pip install pygame
# !pip install keyboard
# !pip install mouse


In [2]:
# Import library
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

import os
import json
import mouse
import ctypes
import pickle
import pygame
import keyboard
import datetime
import pandas as pd
from time import sleep
from getpass import getpass

from google.cloud import bigquery

from api_ig import instagram_id_user, tagged_post
import my_bigquery_function as bq_


def save_dictionary_as_pkl(the_dictionary: dict, fname: str):
    with open(f'{fname}', 'wb') as file:
        pickle.dump(the_dictionary, file)
    return "Saved as .pkl!"

def read_dictionary_pkl(fname: str):
    with open(f'{fname}', 'rb') as file:
        my_dict = pickle.load(file)
    return my_dict

def save_dictionary_as_json(the_dictionary, fname):
    with open(f'{fname}', "w") as file:
        json.dump(the_dictionary, file)
    return "Saved as .json!"

def read_dictionary_json(fname):
    with open(f'{fname}', "r") as file:
        data_dict = json.load(file)
    return data_dict

def display_dictionary(d: dict):
    print(json.dumps(d, indent=4))

def remainder(n:int=2000):
    pygame.mixer.init()
    pygame.mixer.music.load("dj_dayak_malihi_janji.mp3")
    pygame.mixer.music.play()

    pygame.time.delay(n)  # duration
    pygame.mixer.music.stop()
    pygame.mixer.quit()

def get_mouse_position():
    n=1; sleep(3)
    while True:
        print(f"{n}. {mouse.get_position()}")
        n+=1; sleep(3)

position = {
    "instagram_page": [535, 384],
    "inspector_page": [1274, 317],
    "network_panel_tab": [1270, 128],
    "filter": [1070, 175],
    "api": [1054, 340],
    "response": [1402, 323],
    "item_response": [1398, 343],

    "open_ig_feed_responses.json": [275, 45],
    "json_field": [250, 104],
}


pygame 2.5.0 (SDL 2.28.0, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Chrome
from selenium.webdriver.chrome.options import Options

username = str(input("Username: "))
password = getpass("Password: ")
url = "https://www.instagram.com/"

# Configuration
os.environ['PATH'] += r";C:/SeleniumDrivers"
chromedriver_path = "C:/SeleniumDrivers/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver_path, options=Options().add_experimental_option('w3c', False))
wait = WebDriverWait(driver, 20)
driver.maximize_window()


In [4]:
def f12():
    keyboard.press_and_release('f12')

def new_tab():
    driver.execute_script("window.open('about:blank', '_blank');")
    sleep(0.5)

def ctrl_tab():
    driver.switch_to.window(driver.window_handles[2])
    sleep(1)
    
def ctrl_shift_tab():
    driver.switch_to.window(driver.window_handles[1])
    sleep(1)

def smaller_ig_page():
    mouse.move(position['instagram_page'][0], position['instagram_page'][1], duration=0.15)
    keyboard.press("ctrl")
    mouse.wheel(-1)
    sleep(0.2)
    mouse.wheel(-1)
    sleep(0.2)
    mouse.wheel(-1)
    sleep(0.2)
    mouse.wheel(-1)
    keyboard.release("ctrl")

def smaller_inpector_page():
    mouse.move(position['inspector_page'][0], position['inspector_page'][1], duration=0.6)
    keyboard.press("ctrl")
    mouse.wheel(-1)
    keyboard.release("ctrl")
    
def open_network_panel():
    mouse.move(position['network_panel_tab'][0], position['network_panel_tab'][1], duration=0.3)
    mouse.click('left')
    
def filter_api_search():
    mouse.move(position['filter'][0], position['filter'][1], duration=0.3)
    mouse.click('left'); sleep(0.3)
    
    for s in "/api/v1/feed/user":
        keyboard.write(s); sleep(0.1)
    
def copy_response_to_clipboard():
    mouse.move(position['api'][0], position['api'][1], duration=0.2)
    mouse.click("left"); sleep(0.3)
    mouse.move(position['response'][0], position['response'][1], duration=0.2)
    mouse.click("left"); sleep(0.3)
    mouse.move(position['item_response'][0], position['item_response'][1], duration=0.7)
    mouse.double_click("left"); sleep(0.1)
    keyboard.press_and_release("ctrl + a"); sleep(0.08)
    keyboard.press_and_release("ctrl + c")

def paste_to_localfile():
    sleep(0.4); keyboard.press_and_release('win + 4'); sleep(0.4)
    mouse.move(position['open_ig_feed_responses.json'][0], position['open_ig_feed_responses.json'][1], duration=0.3); sleep(0.15)
    mouse.click('left')
    mouse.move(position['json_field'][0], position['json_field'][1], duration=0.3); sleep(0.15)
    mouse.click('left'); sleep(0.2)
    keyboard.press_and_release('ctrl + a'); sleep(0.6)
    keyboard.press_and_release('backspace'); sleep(0.4)
    keyboard.press_and_release('ctrl + v'); sleep(1)
    keyboard.press_and_release('ctrl + s'); sleep(1)
    keyboard.press_and_release('ctrl + home'); sleep(0.2)
    keyboard.press_and_release('win + 5'); sleep(2)
    
def clear_clipboard_history():
    try:
        ctypes.windll.user32.OpenClipboard(None)
        ctypes.windll.user32.EmptyClipboard()
        ctypes.windll.user32.CloseClipboard()
        # print("Clipboard history cleared successfully.")
    except Exception as e:
        print("An error occurred while clearing the clipboard history:", str(e))


## 1. Login IG

In [5]:
def login_ig(driver: webdriver, wait: WebDriverWait, username:str, password:str , url:str="https://www.instagram.com"):
    # Open IG in chrome web browser
    driver.get(url)

    # input username
    input_username = wait.until(EC.visibility_of_element_located((By.NAME, "username")))
    for i in username:
        input_username.send_keys(i)
        sleep(0.13)

    # input password
    input_password = wait.until(EC.visibility_of_element_located((By.NAME, "password")))
    for j in password:
        input_password.send_keys(j)
        sleep(0.13)

    # click login button
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button._acan._acap._acas._aj1-")))
    smaller_ig_page(); sleep(0.5)  #  SWITCH
    login_button.click()

    ########################################
    ##### NOW WE ARE SUCCESS TO LOG IN #####
    ########################################

    # Turn on notif windows? 
    try:
        notnow_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button._a9--._a9_1")))
        sleep(1)
        notnow_button.click()
        
    # Save your login info?
    except TimeoutException:
        notnow_button = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.x1i10hfl.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.xdl72j9.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1q0g3np.x1lku1pv.x1a2a7pz.x6s0dn4.xjyslct.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x9f619.x1ypdohk.x1i0vuye.x1f6kntn.xwhw2v2.xl56j7k.x17ydfre.x2b8uid.xlyipyv.x87ps6o.x14atkfc.xcdnw81.xjbqb8w.xm3z3ea.x1x8b98j.x131883w.x16mih1h.x972fbf.xcfux6l.x1qhh985.xm0m39n.xt0psk2.xt7dq6l.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x1n5bzlp.x173jzuc.x1yc6y37")))
        sleep(1)
        notnow_button.click()        

    sleep(2); new_tab(); new_tab()
    driver.switch_to.window(driver.window_handles[1])
    
    sleep(0.3)
    f12(); sleep(0.4)  #  SWITCH
    smaller_inpector_page(); sleep(0.17)  #  SWITCH
    open_network_panel(); sleep(0.3)  #  SWITCH
    
    filter_api_search()  #  SWITCH


## 2. Scraping IG

In [6]:
instagram_users = {
    'id': [],  # int
    'username': [],  # str
    'display_name': [],  # str
    'profile_picture': [],  # str
    'verified': [],  # bool
    'interest': [],  # str
    'posts': [],  # int
    'followers': [],  # int
    'following': [],  # int
    'tagged_post': [],  # int
    'description': [],  # str
}

user_posts = {
    'id_post': [],  # str
    'taken_at': [],  # timestamp
    'id': [],  # int
    'media_type': [],  # int
    'created_at': [],  # timestamp
    'created_at_utc': [],  # timestamp
    'can_viewer_reshare': [],  # bool
    'caption_is_edited': [],  # bool
    'play_count': [],  # int
    'like_count': [],  # int
    'comment_count': [],  # int
    'video_duration': [],  # float
    'fb_play_count': [],  # int
    'fb_like_count': [],  # int
    'like_and_view_counts_disabled': [],  # 
    'carousel_media_count': [],  # int
    'thumbnail_url': [],  # str
    'caption': [],  # str
}
# instagram_users = read_dictionary_pkl("instagram_users_new.pkl")
# user_posts = read_dictionary_pkl("user_posts_new.pkl")


def scrape_ig(driver: webdriver.Chrome, wait: WebDriverWait, target_ig_usernames: list[str]):
    def username_():
        try:
            scraped_username = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.x1ms8i2q.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')))
            scraped_username = scraped_username.get_attribute('textContent')
        except TimeoutException:
            scraped_username = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.x1ms8i2q.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')))
            scraped_username = scraped_username.get_attribute('textContent')
            
        return scraped_username
    
    def display_name_():
        try:
            scraped_display_name = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span.x1lliihq.x1plvlek.xryxfnj.x1n2onr6.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.x1i0vuye.xvs91rp.x1s688f.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')))
            scraped_display_name = scraped_display_name.get_attribute('textContent')
        except TimeoutException:
            scraped_display_name = "None"
        except NoSuchElementException:
            scraped_display_name = "None"
            
        return scraped_display_name
    
    def posts_followers_following_():
        # Wait parent
        ul_parent = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'ul.x78zum5.x1q0g3np.xieb3on')))  # Wait until the <ul> element is visible

        ######################################################################################################################
        # 1 Post
        li_posts = ul_parent.find_element(By.XPATH, '//li[contains(text(), "posts")]')
        span_element = li_posts.find_element(By.CLASS_NAME, '_ac2a')
        posts_value = int(span_element.get_attribute('textContent').replace(",", ""))
        
        # 2 Followers
        try:    
            li_followers = ul_parent.find_element(By.XPATH, '//li[a[contains(text(), "followers")]]')
            span_element = li_followers.find_element(By.CLASS_NAME, '_ac2a')
            followers_value = int(span_element.get_attribute('title').replace(",", ""))
        except NoSuchElementException:
            li_followers = ul_parent.find_element(By.XPATH, '//li[contains(text(), "followers")]')
            span_element = li_followers.find_element(By.CLASS_NAME, '_ac2a')
            followers_value = int(span_element.get_attribute('title').replace(",", ""))

        # 3 Following
        try:
            li_following = ul_parent.find_element(By.XPATH, '//li[a[contains(text(), "following")]]')
            span_element = li_following.find_element(By.CLASS_NAME, '_ac2a')
            following_value = int(span_element.get_attribute('textContent').replace(",", ""))
        except NoSuchElementException:
            li_following = ul_parent.find_element(By.XPATH, '//li[contains(text(), "following")]')
            span_element = li_following.find_element(By.CLASS_NAME, '_ac2a')
            following_value = int(span_element.get_attribute('textContent').replace(",", ""))
        ######################################################################################################################

        return posts_value, followers_value, following_value

    def profile_picture_(ig_username):
        xpth = f"//img[contains(@class, 'xpdipgo x6umtig') and @alt=\"{ig_username}'s profile picture\"]"
        img_element = driver.find_element_by_xpath(xpth)
        src_value = img_element.get_attribute("src")
        
        return src_value

    def verified_():
        try:
            div_element = driver.find_element_by_class_name('x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1gslohp.x1i64zmx.x1n2onr6.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1')
            is_v = True
        except NoSuchElementException:
            is_v = False
        
        return is_v
    
    def interest_():
        try:
            outer_div = driver.find_element_by_class_name('x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1n2onr6.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1')
            inner_div = driver.find_element_by_class_name('_aacl._aaco._aacu._aacy._aad6._aade')
            inter = inner_div.text
        except NoSuchElementException:
            inter = "None"
            
        return inter
    
    def description_():
        try:    
            scraped_desc = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1._aacl._aaco._aacu._aacx._aad6._aade')))
            scraped_desc = scraped_desc.get_attribute('textContent')
        except TimeoutException:
            scraped_desc = "None"
        except NoSuchElementException:
            scraped_desc = "None"
            
        return scraped_desc
    
    for ig_username in target_ig_usernames:
        # instagram_users = read_dictionary_pkl("instagram_users_new.pkl")
        # user_posts = read_dictionary_pkl("user_posts_new.pkl")
        
        driver.get(url+ig_username)
        
        scraped_username = username_()
        scraped_display_name = display_name_()
        posts, followers, following = posts_followers_following_()
        is_verified = verified_()
        interest = interest_()
        scraped_description = description_()
        profile_picture = profile_picture_(ig_username)
        
        
        
        ############################################################################################################################
        copy_response_to_clipboard()
        
        # FOR pasting, make sure VS Code is on win+4 & chrome on win+5
        # VS code tab (1) ipynb, (2) json
        paste_to_localfile()
        data = json.load(open('ig_feed_responses.json'))
        
        for i in range(data['num_results']):
            user_posts['id_post'].append(str(data['items'][i]['pk']))
            user_posts['taken_at'].append(
                datetime.datetime.fromtimestamp(int(data['items'][i]['taken_at']))
                )
            user_posts['id'].append(int(data['user']['pk']))
            user_posts['media_type'].append(data['items'][i]['media_type'])
            user_posts['can_viewer_reshare'].append(data['items'][i]['can_viewer_reshare'])
            if data['items'][i]['caption'] == None: 
                user_posts['created_at'].append(None)
                user_posts['created_at_utc'].append(None)
                user_posts['caption_is_edited'].append(None)
            else:
                user_posts['created_at'].append(
                    datetime.datetime.fromtimestamp(data['items'][i]['caption']['created_at'])
                    )
                user_posts['created_at_utc'].append(
                    datetime.datetime.utcfromtimestamp(data['items'][i]['caption']['created_at_utc'])
                    )
                user_posts['caption_is_edited'].append(data['items'][i]['caption_is_edited'])
            
            try:
                user_posts['play_count'].append(int(data['items'][i]['play_count']))
            except KeyError:
                user_posts['play_count'].append(None)

            try:
                user_posts['like_count'].append(int(data['items'][i]['like_count']))
            except KeyError:
                user_posts['like_count'].append(None)

            try:
                user_posts['comment_count'].append(int(data['items'][i]['comment_count']))
            except KeyError:
                user_posts['comment_count'].append(None)

            try:
                user_posts['video_duration'].append(data['items'][i]['video_duration'])
            except KeyError:
                user_posts['video_duration'].append(None)

            try:
                user_posts['fb_play_count'].append(int(data['items'][i]['fb_play_count']))
            except KeyError:
                user_posts['fb_play_count'].append(None)

            try:
                user_posts['fb_like_count'].append(int(data['items'][i]['fb_like_count']))
            except KeyError:
                user_posts['fb_like_count'].append(None)

            try:
                user_posts['like_and_view_counts_disabled'].append(data['items'][i]['like_and_view_counts_disabled'])
            except KeyError:
                user_posts['like_and_view_counts_disabled'].append(None)

            try:
                user_posts['carousel_media_count'].append(int(data['items'][i]['carousel_media_count']))
            except KeyError:
                user_posts['carousel_media_count'].append(None)

            user_posts['thumbnail_url'].append(data['items'][i]['image_versions2']['candidates'][0]['url'])

            try:
                user_posts['caption'].append(data['items'][i]['caption']['text'])
            except KeyError:
                user_posts['caption'].append(None)
            except TypeError:
                user_posts['caption'].append(None)
        ############################################################################################################################
        
        
        
        sleep(0.7); ctrl_tab()
        ig_id = instagram_id_user(driver, wait, ig_username)
        num_of_tag = tagged_post(driver, wait, ig_id)
        # num_of_tag = float('-inf')
        
        # Store scraped data
        instagram_users['id'].append(ig_id)
        instagram_users['username'].append(scraped_username)
        instagram_users['display_name'].append(scraped_display_name)
        instagram_users['profile_picture'].append(profile_picture)
        instagram_users['verified'].append(is_verified)
        instagram_users['interest'].append(interest)
        instagram_users['posts'].append(posts)
        instagram_users['followers'].append(followers)
        instagram_users['following'].append(following)
        instagram_users['tagged_post'].append(num_of_tag)
        instagram_users['description'].append(scraped_description)
        
        clear_clipboard_history()
        
        # save_dictionary_as_pkl(instagram_users, "instagram_users_new.pkl")
        # save_dictionary_as_pkl(user_posts, "user_posts_new.pkl")
        
        # display(pd.DataFrame(instagram_users))
        sleep(0.75)
        ctrl_shift_tab()
        sleep(2)

    return pd.DataFrame(instagram_users), pd.DataFrame(user_posts)


In [7]:
list_username = [
    'brendapantja',
    'ireneverent',
    'tasyafarasya',
    'mawar_eva',
    'kayla.nadira',
    'real.hanummegaa',
    'bellagap',
    'saritiw',
    'rachgoddard',
    'nandaarsynt',
    'siscamci6',
    'dewithsri',
    'christomci6',
    'brigittairene22',
    'cintiacicin',
    'gohana_navra',
    'karencarlotta',
    'mgdalenaf',
    'maulinabalqiis',
    'moona_jingga',
    'claraayusheila',
    'diniyaan',
    'inkangela',
    'jaclynariandra',
    'joviadhiguna',
    'lulalahfah',
    'adindathomas',
    'annisanndr',
    'anazsiantar',
    'ridwanhr',
    'fitra.eri',
    'motomobitv',
    'maswahid',
    'bylizzieparra',
    'jenniferbachdim',
    'aulion',
    'jeromepolin',
    'bintangemon',
    'kristo.immanuel',
    'keanuagl',
    'daraarafah',
    'anyageraldine',
    'rizkyfbian',
    'fiersabesari',
    'iqbaal.e',
    'jefrinichol',
    'dr.tirta',
    'onadioleonardo_official',
    'aurelie',
    'danillariyadi',
]

login_ig(driver, wait, username, password)
instagram_users_new, user_posts_new = scrape_ig(driver, wait, list_username)
remainder()


In [19]:
# Saved tagged post
# instagram_users_new = read_dictionary_pkl('instagram_users_new.pkl')
# user_posts_new = read_dictionary_pkl('user_posts_new.pkl')

# print(instagram_users_new['tagged_post'])
# instagram_users_new['tagged_post'] = [981, 12, 65478, 10884, 2041, 2836, 36474, 1121, 11536, 4722, 831, 266, 523, 94, 248, 57, 1999, 12790, 48, 929, 1046, 11090, 1151, 1966, 12187, 9240, 3821, 73, 8759, 5320, 26000, 23937, 4633, 6106, 9182, 2297, 47687, 3079, 230, 57437, 13108, 109798, 27694, 89744, 73877, 54711, 46759, 10966, 207, 32978]
# print(instagram_users_new['tagged_post'])
# save_dictionary_as_pkl(instagram_users_new,'instagram_users_new.pkl')


[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]
[981, 12, 65478, 10884, 2041, 2836, 36474, 1121, 11536, 4722, 831, 266, 523, 94, 248, 57, 1999, 12790, 48, 929, 1046, 11090, 1151, 1966, 12187, 9240, 3821, 73, 8759, 5320, 26000, 23937, 4633, 6106, 9182, 2297, 47687, 3079, 230, 57437, 13108, 109798, 27694, 89744, 73877, 54711, 46759, 10966, 207, 32978]


'Saved as .pkl!'

In [22]:
# BigQuery
instagram_users_new = pd.DataFrame(read_dictionary_pkl("instagram_users_new.pkl"))
user_posts_new = pd.DataFrame(read_dictionary_pkl("user_posts_new.pkl"))

cred = "C:\\Users\\victo\\[Internship] Data Labs\\dla-internship-program.json"

schema1 = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("username", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("display_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("profile_picture", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("verified", "BOOL", mode="NULLABLE"),
    bigquery.SchemaField("interest", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("posts", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("followers", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("following", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("tagged_post", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
]
schema2 = [
    bigquery.SchemaField("id_post", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("taken_at", "DATETIME", mode="NULLABLE"),
    bigquery.SchemaField("id", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("media_type", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("created_at", "DATETIME", mode="NULLABLE"),
    bigquery.SchemaField("created_at_utc", "DATETIME", mode="NULLABLE"),
    bigquery.SchemaField("can_viewer_reshare", "BOOL", mode="NULLABLE"),
    bigquery.SchemaField("caption_is_edited", "BOOL", mode="NULLABLE"),
    bigquery.SchemaField("play_count", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("like_count", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("comment_count", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("video_duration", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("fb_play_count", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("fb_like_count", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("like_and_view_counts_disabled", "BOOL", mode="NULLABLE"),
    bigquery.SchemaField("carousel_media_count", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("thumbnail_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("caption", "STRING", mode="NULLABLE"),
]

if bq_.dataset_exists(cred, "scraping_instagram") == "NotFound":
    bq_.create_dataset(cred, "scraping_instagram")
    bq_.create_table(cred, "scraping_instagram", "instagram_users_new", schema1)
    bq_.create_table(cred, "scraping_instagram", "user_posts_new", schema2)

bq_.load_table_dataframe(cred, "dla-internship-program.scraping_instagram.instagram_users", instagram_users_new)
bq_.load_table_dataframe(cred, "dla-internship-program.scraping_instagram.user_posts", user_posts_new)
print(f'{datetime.date.today().strftime("%B %d, %Y")}\t{datetime.datetime.now().strftime("%H:%M:%S")}')


Dataset dla-internship-program.scraping_instagram already exists
Loaded 50 rows and 11 columns to dla-internship-program.scraping_instagram.instagram_users
Loaded 660 rows and 18 columns to dla-internship-program.scraping_instagram.user_posts
July 31, 2023	09:37:08


In [ ]:
instagram_users_new = pd.DataFrame(read_dictionary_pkl("instagram_users_new.pkl"))
user_posts_new = pd.DataFrame(read_dictionary_pkl("user_posts_new.pkl"))

In [ ]:
instagram_users_new

,id,username,display_name,profile_picture,verified,interest,posts,followers,following,tagged_post,description
0,205382497,brendapantja,Brenda Immanuella ✩,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,False,Blogger,1209,33461,3963,981,♡ content + hosting — jkt/ bali ♡ fashion + ac...
1,180554536,ireneverent,None,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,False,None,60,14263,1156,12,"jkt, id.🛒 @ilen.po @iryson.id -for business in..."
2,174788501,tasyafarasya,Tasya Farasya,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,None,2842,6287220,933,65478,🦋@mop.beauty @goldenblack.coffee
3,1385796808,mawar_eva,Mawar Eva de Jongh,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,None,506,2113607,1322,10884,BA of @emeronhaircare @mustikaratubright @saha...
4,2053858170,kayla.nadira,Annisa Kayla Nadira,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,None,72,1142802,889,2041,@kazami_store @kyl.cosmetic @nareforher @shaw...
5,4075959271,real.hanummegaa,Hanummegaa,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,Entrepreneur,546,2110682,769,2836,business collaboration 08111877480 Dina@hanumb...
6,209254407,bellagap,Bella Graceva Amanda Putri,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,Public figure,553,1484449,1352,36474,Business Inquiries :bellagap.business@gmail.co...
7,547295334,saritiw,Sari Endah Pratiwi,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,Digital creator,4019,1000494,1334,1121,✨@shirazaffa✨—Business Inquiries:• brand campa...
8,238369871,rachgoddard,Rachel Goddard,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,Public figure,4276,1164494,1150,11536,"CONTENT CREATOR (BEAUTY, TRAVEL & LIFESTYLE)GA..."
9,25982025,nandaarsynt,Nanda Arsyinta ʕ •ᴥ•ʔ,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,True,Public figure,1605,3174428,546,4722,𝚏𝚊𝚜𝚑𝚒𝚘𝚗 • 𝚋𝚎𝚊𝚞𝚝𝚢 • 𝚝𝚛𝚊𝚟𝚎𝚕BA @mamaschoiceid💌 na...


In [ ]:
user_posts_new

,id_post,taken_at,id,media_type,created_at,created_at_utc,can_viewer_reshare,caption_is_edited,play_count,like_count,comment_count,video_duration,fb_play_count,fb_like_count,like_and_view_counts_disabled,carousel_media_count,thumbnail_url,caption
0,3154015487227555177,2023-07-24 21:12:47,205382497,8,2023-07-24 21:12:48,2023-07-24 14:12:48,True,False,NaN,3,25,NaN,NaN,NaN,True,10.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,☁️☁️☁️ wrapped in @_kerokoo \n\n#BrensBaliDiary
1,3153892694380887382,2023-07-24 17:08:49,205382497,8,2023-07-24 17:09:06,2023-07-24 10:09:06,True,True,NaN,3,6,NaN,NaN,NaN,True,10.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,Island diary ✩ \n\n#BrensBaliDiary
2,3153250184671354403,2023-07-23 19:52:16,205382497,8,2023-07-23 19:53:38,2023-07-23 12:53:38,True,True,NaN,3,4,NaN,NaN,NaN,True,10.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,"✩!!! Guys, let me tell you about Peachy in Can..."
3,3153028842701240742,2023-07-23 12:32:30,205382497,8,2023-07-23 12:32:31,2023-07-23 05:32:31,True,False,NaN,3,17,NaN,NaN,NaN,True,6.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,(Not so) slow morning in Bali — lounging in @e...
4,3151638801894643592,2023-07-21 14:30:44,205382497,8,2023-07-21 14:30:46,2023-07-21 07:30:46,True,False,NaN,3,9,NaN,NaN,NaN,True,10.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,Throwback to an unforgettable self-care day wi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,3133037691633769030,2023-06-25 22:33:39,1261917618,8,2023-06-25 22:33:40,2023-06-25 15:33:40,True,False,NaN,66577,145,NaN,NaN,NaN,False,8.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,Intim sekali kita kemarin gaes 🥰❤️\n\nPhoto by...
656,3131509002723009087,2023-06-23 19:56:25,1261917618,8,2023-06-23 19:56:26,2023-06-23 12:56:26,True,False,NaN,43378,61,NaN,NaN,NaN,False,3.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,Abis laundry besok manggung!! Sampai jumpa di ...
657,3130097527881818604,2023-06-21 21:14:00,1261917618,2,2023-06-21 21:16:36,2023-06-21 14:16:36,True,True,211821.0,12656,34,97.755,NaN,NaN,False,NaN,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,Akhirnya gue punya waktu lagi untuk cover sala...
658,3128558963776917374,2023-06-19 18:15:13,1261917618,8,2023-06-19 18:15:14,2023-06-19 11:15:14,True,False,NaN,33934,109,NaN,NaN,NaN,False,7.0,https://instagram.fcgk6-2.fna.fbcdn.net/v/t51....,"Terima kasih Bandung, untuk kalian semuanya ya..."


In [ ]:
# # Save to .csv
instagram_users_new.to_csv('instagram_users_new.csv', index=False)
user_posts_new.to_csv('user_posts_new.csv', index=False)
